In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import glob
import os

## Setup

In [ ]:
def create_directory(head_directory_name, structure_number):
    directory = os.path.join(head_directory_name, str(structure_number))
    os.makedirs(directory)
    return directory

In [ ]:
# Define number of structures
num_struct_in_fit = 1 #How many structures are in the fit

# Create cross validation directory
head_directory_name = '{}_structure_fits'.format(num_struct_in_fit)
# head_directory_name = 'scale_testing/scale_0.1'
plot_directory_name = 'plots'
head_output_directory = create_directory(head_directory_name, plot_directory_name)

## This section plots the errors of each structure
Firstly the errors are read in from the sub directories into a dictionary, converting the error to a float and using the directory path as the structure number by removing the error file extention. This won't work for a directory tree, only for a depth of 1.

In [ ]:
error_dict = {}
for error_file in sorted(glob.glob('{}/*/error.dat'.format(head_directory_name))):
    error = float(np.loadtxt(error_file))
    structure_num = error_file.replace('/error.dat', '').replace('{}/'.format(head_directory_name), '')
    error_dict.update( {structure_num:error}) #Remove int if directory naming system changes to include strings. This will change the order of the structures though.

In [ ]:
plt.scatter(*zip(*sorted(error_dict.items())))
plt.xticks(rotation=50)
plt.xlabel('structure numbers')
plt.ylabel('$\chi^2$ error')
plt.title('{} structure fit errors ($\chi^2$)'.format(num_struct_in_fit))
plt.savefig('{}/{}_structure_fit_errors.png'.format(head_output_directory, num_struct_in_fit),dpi=500, bbox_inches = "tight")
plt.show()

## This section plots the separate potential parameters for each structure
Firstly the potentials are read in as a dictionary from each structures potentials.json file, these are edited to include the structure number, then appended to a list of potential dictionaries. These dictionaries are then combined to have all structures in one dictionary, where the keys are the potential name and the values are the tuples containing structure number and value. These are then plotted in individual plots by potential.

In [ ]:
list_of_dict = []
for potential_file in sorted(glob.glob('{}/*/potentials.json'.format(head_directory_name))):
    with open(potential_file, 'r') as f:
        potentials = json.load(f)
    structure_num = potential_file.replace('/potentials.json', '').replace('{}/'.format(head_directory_name), '')
    potentials.update((k, (structure_num,v) ) for k,v in potentials.items()) #Remove int if directory naming system changes to include strings. This will change the order of the structures though.
    list_of_dict.append(potentials)

In [ ]:
potentials_dict = {}
for key in list_of_dict[0].keys():
    potentials_dict[key] = [potentials_dict[key] for potentials_dict in list_of_dict]

In [ ]:
for k,v in potentials_dict.items():
    x_val = [x[0] for x in v]
    y_val = [y[1] for y in v]
    plt.scatter(x_val,y_val)
    plt.xticks(rotation=50)
    plt.xlabel('structure number')
    plt.ylabel('{} value'.format(k))
    plt.title('{} {} structure fit'.format(k, num_struct_in_fit))
    plt.savefig('{}/{}_structure_fit_{}.png'.format(head_output_directory, num_struct_in_fit, k) ,dpi=500, bbox_inches = "tight")
    plt.show()

## Plot forces and stresses

In [ ]:
structure_num = 15
for structure in sorted(glob.glob('{}/{}'.format(head_directory_name, structure_num))):
    dft_forces = np.loadtxt('{}/dft_forces.dat'.format(structure), delimiter=' ')
    ip_forces = np.loadtxt('{}/ip_forces.dat'.format(structure), delimiter=' ')
    dft_stresses = np.loadtxt('{}/dft_stresses.dat'.format(structure), delimiter=' ')
    ip_stresses = np.loadtxt('{}/ip_stresses.dat'.format(structure), delimiter=' ')

In [ ]:
x= np.arange(0, len(dft_forces.flatten()))
plt.scatter(x, dft_forces.flatten(), label='dft')
plt.scatter(x, ip_forces.flatten(), label='ip')
plt.legend()
# plt.xticks(rotation=50)
plt.xlabel('atom number')
plt.ylabel('force')
plt.text(0,3.5, 'error: {0:.5f}'.format(np.sum((dft_forces - ip_forces)**2)/ dft_forces.size))
plt.savefig('{}/forces.png'.format(head_output_directory),dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
x= ['XX', 'YY', 'ZZ', 'XY', 'YZ', 'ZX'] 
plt.scatter(x, dft_stresses.flatten(), label='dft')
plt.scatter(x, ip_stresses.flatten(), label='ip')
plt.legend()
# plt.xticks(rotation=50)
plt.ylabel('stress')
plt.text(0,-10, 'error: {0:.5f}'.format(np.sum((dft_stresses - ip_stresses)**2)/ dft_stresses.size))
plt.savefig('{}/stresses.png'.format(head_output_directory),dpi=500, bbox_inches = "tight")
plt.show()

In [ ]:
force_diff = np.sum((dft_forces - ip_forces)**2)/ dft_forces.size
stress_diff = np.sum((dft_stresses - ip_stresses)**2)/ dft_stresses.size
force_diff+(stress_diff*0.1)